In [16]:
from IPython.core.display import HTML


##1.Introduction

<img  src="images/RNN/BR7I3EWF4TEXTAN9VL8OK4OQXFC61N1W.png" width=75%/>

Each rectangle is a vector and arrows represent functions (e.g. matrix multiply). Input vectors are in red, output vectors are in blue and green vectors hold the RNN's state (more on this soon). From left to right: (1) Vanilla mode of processing without RNN, from fixed-sized input to fixed-sized output (e.g. image classification). (2) Sequence output (e.g. image captioning takes an image and outputs a sentence of words). (3) Sequence input (e.g. sentiment analysis where a given sentence is classified as expressing positive or negative sentiment). (4) Sequence input and sequence output (e.g. Machine Translation: an RNN reads a sentence in English and then outputs a sentence in French). (5) Synced sequence input and output (e.g. video classification where we wish to label each frame of the video). Notice that in every case are no pre-specified constraints on the lengths sequences because the recurrent transformation (green) is fixed and can be applied as many times as we like.

###1.1.Sequential processing in absence of sequences

The figure below shows results from two very nice papers from **DeepMind**. On the left, an algorithm learns a recurrent network policy that steers its attention around an image; In particular, it learns to read out house numbers from left to right ([Multiple Object Recognition with Visual Attention](http://arxiv.org/abs/1412.7755)). On the right, a recurrent network generates images of digits by learning to sequentially add color to a canvas ([DRAW: A Recurrent Neural Network For Image Generation](http://arxiv.org/abs/1502.04623)):

<div class="imgcap" style="color: #555;font-size: 14px;text-align: center;">
<div>
<img src="images/RNN/house_read.gif" style="margin-top:12px; max-width:49%; height:400px; float:left; ">
<img src="images/RNN/house_generate.gif" style="max-width:49%; height:400px; float: left">
</div>
</div>


##2.Recurrent Neural Networks (RNN)

Figure below introduce the RNN architecture where rectangular box is a hidden layer at a time-step $t$. At time $t$. nodesre ceiving input along recurrent edges receive **input activation** from the current example $x_t$ and also from **hidden nodes** $h_{t-1}$ int the networks's previous state. The output $\hat{y}_t$ is calculated given the hidden state $h_t$ at that time step.

$$h_t = \sigma(W^{hh}h_{t-1} + W^{hx}x_t + b_h)$$

$$\hat{y}_t = softmax(W^{yh}h_t + b_y)$$

Here $W^{hx}$ is the matrix weights between the input and hidden layers and $W^{hh}$ is the matrix of recurrent weights between the hidden layers at adjacent time steps.

<img  src="images/RNN/UB4ORE2T1YFBLATH7C3IJH6BUQRSHCK9.png" height="50%"/>

<img  src="images/RNN/T5H7N9M2NUU4BT80EFHOJCQJIADODUBL.png"/>

<img  src="images/RNN/ONWMIE8HWMBPTT3UWO54YJTKGW7RHG8N.png"/>

<img  src="images/RNN/DMU4YPNQGERMHEWCMVLPGM3JJ90A2TYT.png"/>

###2.1.The vanishing/exploding gradient problem
Recurrent neural networks propagate weight matrices from one timestep to the next. Recall the goal of a RNN implementation is to enable propagating context information through faraway time-steps.

Consider the following two sentences:
+ **Jane walked into the room. John walked in too. Jane said hi to ___**
+ **Jane walked into the room. John walked in too. It was late in the day, and everyone was walking home after a long day at work. Jane said hi to ___**

In practice, it turns out RNNs are more likely to correctly predict the blank spot in Sentence 1 than in Sentence 2. This is because during the back-propagation phase, the contribution of gradient values gradually vanishes as they propagate to earlier time steps. 

During experimentation, once the gradient value grows extremely large, it causes an overflow (i.e. NaN) which is easily detectable at
runtime; this issue is called the **Gradient Explosion Problem**. When the gradient value goes to zero, however, it can go undetected while drastically reducing the learning quality of the model for far-away words in the corpus; this issue is called the **Vanishing Gradient Problem**.

###2.2.Solution to the Exploding Gradients
The exploding gradients problem is relatively easy to handle. Thomas Mikolov first introduced a simple heuristic solution that **clips** gradients to a small number whenever they explode. That is, whenever they reach a certain threshold, they are set back to a small number as shown below.

<img  src="images/RNN/KGRO19FN3I2VW0DKRX1EM4XBBXQE6AQ8.png" width=75%/>

###2.3.Long Short-Term Memory Unit and Gated Recurrent Unit
[Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling](http://arxiv.org/abs/1412.3555)
<img  src="images/RNN/F3CUNU9KS02DNKRLD1H23VF4PI6P806X.png" width=75%/>

####Long Short-Term Memory Unit

The Long Short-Term Memory (LSTM) unit was initially proposed by Hochreiter and Schmidhuber[1997]. Since then, a number of minor modifications to the original LSTM unit have been made.We follow the implementation of LSTM as used in Graves [2013].

Each $j$-th LSTM unit maintains a **memory** $c_t^j$ at time $t$. The output $h_t^j$ is then

$$h_t^j = o_t^j tanh(c_t^j)$$

where $o_t^j$ is an **output gate** and computed by

$$o_t^j = \sigma (W_ox_t + U_oh_{t-1} + V_oc_t)^j$$

where $V_o$ is a diagonal matrix.

The memory cell $c_t^j$ is updated by partially **forgetting** the existing memory and adding a new memory content $\widetilde{c}_t^j$

$$c_j^t = f_t^j c_{t-1}^j + i_t^j \widetilde{c}_t^j$$

where the new memory content is 

$$\widetilde{c}_t^j = tanh (W_cx_t + U_c h_{t-1})^j$$

The extent to which the existing memory is forgotten is modulated by a **forget gate** $f_t^j$, and the degree to which the new memory content is added to the memory cell is modulated by an **input gate** $i_t^j$

$$f_t^j = \sigma (W_fx_t + U_fh_{t-1} +V_fc_{t-1})^j$$

$$i_t^j = \sigma (W_ix_t + U_ih_{t-1} + V_ic_{t-1})$$

####Gated Recurrent Unit
A gated recurrent unit (GRU) was proposed by Cho et al. [2014] to make each recurrent unit to adaptively capture dependencies of different time scales.

The activation $h_t^j$ of the GRU at time $t$ is a linear interpolation between the previous activation $h_{t-1}^j$ and the candidate activation $\widetilde{h}_t^j$

$$h_t^j = (1-z_t^j)h_{t-1}^j + z_t^j\widetilde{h}_t^j$$

where an **update gate** $z_t^j$ decides how much the unit updates its activation, and is conputed by

$$z_t^j = \sigma (W_zx_t +U_zh_{t-1})^j$$

The candidate activation $\widetilde{h}_t^j$ is computed by

$$\widetilde{h}_t^j = tanh (Wx_t + U(r_t \odot h_{t-1}))^j$$

where $r_t$ is a set of **reset gates**. When off ($r_t^j$ cloase to 0),the reset gate effectively makes the unit act as if it is reading the first symbol of an input sequence,allowing it to **forget** the previously computed state

the reset gate $r_t^j$ is computed by:

$$r_t^j = \sigma (W_rx_t + U_rh_{t-1})^j$$

####Advantages of LSTM and GRU
The most prominent feature shared between these units is the additive component of their update from $t$ to $t+1$, which is lacking in the traditional recurrent unit.The traditional recurrent unit always replaces the activation, or the content of a unit with a new value computed from the current input and the previous hidden state. On the other hand, both LSTM unit and GRU keep the existing content and add the new content on top of it.

This additive nature has two advantages:
+ It is easy for each unit to remember the existence of a specific feature in the input stream for a long series of steps.
+ this addition effectively creates shortcut paths that bypass multiple temporal steps. These shortcuts allow the error to be back-propagated easily without too quickly vanishing (if the gating unit is nearly saturated at 1) as a result of passing through multiple, bounded nonlinearities, thus reducing the difficulty due to vanishing gradients

####Experiments
<img  src="images/RNN/UH4R6I1259Q7HY18KDJWHOBT9WN4X6X5.png" width=75%/>

<img  src="images/RNN/A341LCH22LSNMUB5L2JMLPKG1BRNDEX7.png" width=75%/>

####A simple way
[A Simple Way to Initialize Recurrent Networks of Rectified Linear Units](http://arxiv.org/abs/1504.00941)

We initialize the recurrent weight matrix to be the **identity matrix** and biases to be zero. This means that each new hidden state vector is obtained by simply copying the previous hidden vector then adding on the effect of the current inputs and replacing all negative states by zero. In the absence of input, an RNN that is composed of **ReLUs** and initialized with the identity matrix (which we call an IRNN) just stays in the same state indefinitely.

The identity initialization has the very desirable property that when the error derivatives for the hidden units are backpropagated through time they remain constant provided no extra error-derivatives are added. This is the same behavior as LSTMs when their forget gates are set so that there is no decay and it makes it easy to learn very long-range temporal dependencies.

##3.Application
###ReNet
[ReNet: A Recurrent Neural Network Based Alternative to Convolutional Networks](http://arxiv.org/abs/1505.00393)
<img  src="images/RNN/C8AF0KC649RYJS0N73T5CWSOL2RRC5CU.png" width=50%/>
<img  src="images/RNN/X33ES6BGY4ANOPYVWQ0J5WIO9WC4QMTQ.png" width=75%/>

###RNNLM
[Recurrent neural network based language model](http://www.rnnlm.org/)
<img  src="images/RNN/SS8KBCHNKSMS6PK85SWN4512H1IRATL2.png" width=50%/>